In [ ]:
import pandas as pd
from seeq import spy
import livef1

In [ ]:
# Constants
WORKBOOK_NAME = None
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'
CURRENT_SEASON = 2025


In [ ]:

def get_session_data(current_time):
    """Fetch session data from Seeq."""
    posted_session_condition = spy.search({'Name': SESSION_CONDITION_NAME, 'Datasource Name': DATASOURCE_NAME})
    posted_sessions = spy.pull(posted_session_condition, start='2023-01-01T00:00:00Z', end=current_time)
    return posted_sessions[posted_sessions['Capsule End'] < current_time]

def filter_sessions(sessions_df, posted_sessions):
    """Filter sessions that have not been processed yet."""
    return sessions_df[~sessions_df['session_key'].isin(posted_sessions['session_key'])]

def process_telemetry_data(telemetry_data, current_season, current_team_name, current_driver_no):
    """Process telemetry data for a specific driver."""
    telemetry_data = telemetry_data.dropna(subset=['DriverNo'])
    telemetry_data['DriverNo'] = pd.to_numeric(telemetry_data['DriverNo'], errors='coerce').astype('Int64')
    telemetry_data = telemetry_data.drop(columns=['SessionKey', 'DriverNo', 'timestamp'])
    telemetry_data['Utc'] = pd.to_datetime(telemetry_data['Utc'])
    telemetry_data = telemetry_data.set_index('Utc')

    # Rename columns
    telemetry_data.columns = [
        f"{current_season}.{current_team_name}.{current_driver_no}.{col}" for col in telemetry_data.columns
    ]
    return telemetry_data

def process_laps_data(laps_data, current_team_name, current_driver_name):
    """Process laps data for a specific driver."""
    laps_data = laps_data.dropna(subset=['lap_time'])
    laps_data['team_name'] = current_team_name
    laps_data['current_driver_name'] = current_driver_name
    return laps_data

def create_sector_capsules(laps_data, sector_number, start_col, end_col):
    """Create sector capsules for a specific sector."""
    sector_caps = laps_data.copy()
    sector_caps['Capsule Start'] = sector_caps[start_col]
    sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps[end_col]
    sector_caps['Sector'] = sector_number
    sector_caps = sector_caps[['lap_number', 'in_pit', 'DriverNo', 'Capsule Start', 'Capsule End', 'Sector', 'team_name', 'current_driver_name']]
    return sector_caps.dropna(subset=['Capsule Start', 'Capsule End'])

def push_telemetry_data(telemetry_data):
    """Push telemetry data to Seeq."""
    spy.push(data=telemetry_data, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def push_sector_data(all_sectors, current_season, current_team_name, current_driver_no):
    """Push sector data to Seeq."""
    metadata = pd.DataFrame([{
        'Name': f"{current_season}.{current_team_name}.{current_driver_no}.Sectors",
        'Type': 'Condition',
        'Maximum Duration': '5 min',
    }])
    spy.push(data=all_sectors, metadata=metadata, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def process_session(session, car_number_info, current_season):
    """Process a single session."""
    laps_data = session.get_laps()
    telemetry_data = session.get_car_telemetry()

    cars_in_session = telemetry_data.DriverNo.unique()
    cars_pushed = 0

    for current_driver_no in cars_in_session:
        matching_row = car_number_info[
            (car_number_info['MinSessionKey'] <= session.session_key) &
            (car_number_info['MaxSessionKey'] >= session.session_key) &
            (car_number_info['DriverNo'] == current_driver_no)
        ]

        current_team_name = matching_row['TeamName'].values[0] if not matching_row.empty else None
        current_driver_name = matching_row['DriverName'].values[0] if not matching_row.empty else None

        # Process telemetry data
        driver_telemetry_data = process_telemetry_data(telemetry_data[telemetry_data.DriverNo == current_driver_no], current_season, current_team_name, current_driver_no)
        push_telemetry_data(driver_telemetry_data)

        # Process laps data
        driver_laps_data = process_laps_data(laps_data[laps_data.DriverNo == current_driver_no], current_team_name, current_driver_name)

        # Create sector capsules
        sector_1_caps = create_sector_capsules(driver_laps_data, 1, 'lap_start_date', 'sector1_time')
        sector_2_caps = create_sector_capsules(driver_laps_data, 2, 'lap_start_date', 'sector1_time' + 'sector2_time')
        sector_3_caps = create_sector_capsules(driver_laps_data, 3, 'lap_start_date', 'sector1_time' + 'sector2_time' + 'sector3_time')

        all_sectors = pd.concat([sector_1_caps, sector_2_caps, sector_3_caps], ignore_index=True)
        push_sector_data(all_sectors, current_season, current_team_name, current_driver_no)

        cars_pushed += 1

    return cars_pushed



In [ ]:
"""Main function to process sessions."""
current_time = pd.Timestamp.now(tz='UTC')
posted_sessions = get_session_data(current_time)

# Load season and car information
season = livef1.get_season(CURRENT_SEASON)
season.parse_sessions()
sessions_df = season.season_table
car_number_info = pd.read_csv("CarTeamMap.csv")

filtered_sessions = filter_sessions(sessions_df, posted_sessions)

for index, row in filtered_sessions.iterrows():
    session = livef1.get_session(season=CURRENT_SEASON, meeting_key=index, session_key=row['session_key'])
    try:
        session.generate(silver=True)
        cars_pushed = process_session(session, car_number_info, CURRENT_SEASON)
        if cars_pushed > 0:
            # Push session metadata
            session_capsule = pd.DataFrame([row]).reset_index()
            session_capsule = session_capsule.rename(columns={'index': 'meeting_key'})
            spy.push(data=session_capsule, metadata=pd.DataFrame([{
                'Name': SESSION_CONDITION_NAME,
                'Type': 'Condition',
                'Maximum Duration': '7d',
            }]), workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)
    except Exception as e:
        print(f"Skipping session {row['session_key']} due to error: {e}")
